In [2]:
import pdb
#import tensorflow as tf
from keras.models import Sequential,Model
from keras import layers,optimizers
import numpy as np
import pylab as pl
import pandas as pd

from sklearn.datasets import california_housing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale

%matplotlib inline

In [3]:
california=california_housing.fetch_california_housing()

In [4]:
x=california['data']
y=california['target']

x=(x-np.mean(x,axis=0))/np.std(x,axis=0)
y=(y-np.mean(y))/np.std(y)

#df=pd.DataFrame(x,columns=california['feature_names'])

In [5]:
# Splitting into train and test sets

xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.1)

# Random Forest regressor

* Benchmark system
* RF generally gives great performance for problems like this

In [6]:
# MSE Error
rf=RandomForestRegressor(n_estimators=100).fit(xtrain,ytrain)
np.mean((rf.predict(xtest)-ytest)**2)

0.16563101093067775

# "Deep" Neural Network

* Two hidden layers
* Investigate the effect of different params

In [15]:
# Sequential API

net=Sequential()

net.add(layers.Dense(256,input_shape=(8,)))
net.add(layers.Activation('relu'))
net.add(layers.Dropout(rate=.2))
for nUnits in [160,64]:
    net.add(layers.Dense(nUnits,input_shape=(8,)))
    net.add(layers.Activation('relu'))
    #net.add(layers.BatchNormalization())
    net.add(layers.Dropout(rate=.2))
net.add(layers.Dense(1))

opt=optimizers.Adam(lr=.01,decay=.1)
net.compile(opt,loss='mse')

net.fit(xtrain,ytrain,batch_size=1000,validation_data=(xtest,ytest),epochs=500)

Train on 18576 samples, validate on 2064 samples
Epoch 1/500
18576/18576 [==============================] - 0s - loss: 0.5788 - val_loss: 0.4026
Epoch 2/500
18576/18576 [==============================] - 0s - loss: 0.3774 - val_loss: 0.2932
Epoch 3/500
18576/18576 [==============================] - 0s - loss: 0.3412 - val_loss: 0.2654

In [ ]:
# Functional API

inp=layers.Input(shape=(8,))
l1=layers.Dense(256)
d1=l1(inp)
d1r=layers.Activation('relu')(d1)
d1r=layers.Dropout(rate=.2)(d1r)
d2=layers.Dense(160)(d1r)
d2r=layers.Activation('relu')(d2)
d2r=layers.Dropout(rate=.2)(d2r)
d3=layers.Dense(64)(d2r)
d3r=layers.Activation('relu')(d3)
d3r=layers.Dropout(rate=.2)(d3r)
predictions=layers.Dense(1)(d3r)

mod=Model(inputs=inp,outputs=predictions)
mod.compile(optimizers.Adam(lr=.01,decay=.1),loss='mse')
mod.fit(xtrain,ytrain,batch_size=1000,validation_data=(xtest,ytest),epochs=100)